In [3]:
from db_tools.setup import setup
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [4]:
from functools import reduce,partial
from pyspark.sql import functions as F

In [5]:
spark = setup()

23/10/30 14:02:50 WARN Utils: Your hostname, luan-Dell-G15-5520 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/10/30 14:02:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/30 14:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/30 14:02:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


SparkSession available as "spark"


# Tip 1: Alias is your friends (AS is Good practice)

In [6]:
query1 = """SELECT
	customer_id,
	first_name
FROM
	customer
"""
query2 = """SELECT customer_id,amount,payment_date FROM payment"""
df1 = spark.sql(query1)
df2 = spark.sql(query2)

In [7]:
df1.join(df2,on = ['customer_id'], how = 'inner')\
   .orderBy('payment_date',ascending = False)\
   .show()

+-----------+----------+------+--------------------+
|customer_id|first_name|amount|        payment_date|
+-----------+----------+------+--------------------+
|         28|   Cynthia|  2.99|2007-05-14 13:44:...|
|        111|    Carmen|  2.99|2007-05-14 13:44:...|
|        497|   Gilbert|  4.99|2007-05-14 13:44:...|
|        496|     Tyler|  2.99|2007-05-14 13:44:...|
|        596|   Enrique|  0.99|2007-05-14 13:44:...|
|        516|     Elmer|  0.00|2007-05-14 13:44:...|
|        300|      John|  4.99|2007-05-14 13:44:...|
|         53|   Heather|  7.98|2007-05-14 13:44:...|
|        412|     Allen|  0.99|2007-05-14 13:44:...|
|        296|    Ramona|  2.99|2007-05-14 13:44:...|
|        587|    Sergio|  0.99|2007-05-14 13:44:...|
|        115|     Wendy|  2.99|2007-05-14 13:44:...|
|         44|     Marie|  4.99|2007-05-14 13:44:...|
|        597|   Freddie|  4.99|2007-05-14 13:44:...|
|        192|    Laurie|  4.99|2007-05-14 13:44:...|
|        155|      Gail|  7.98|2007-05-14 13:4

In [8]:
df1.join(F.broadcast(df2),on = ['customer_id'], how = 'inner')\
   .orderBy('payment_date',ascending = False)\
   .show()

+-----------+----------+------+--------------------+
|customer_id|first_name|amount|        payment_date|
+-----------+----------+------+--------------------+
|         41| Stephanie|  2.99|2007-05-14 13:44:...|
|         64|    Judith|  4.99|2007-05-14 13:44:...|
|          5| Elizabeth|  0.99|2007-05-14 13:44:...|
|         42|   Carolyn|  0.00|2007-05-14 13:44:...|
|         60|   Mildred|  0.00|2007-05-14 13:44:...|
|         42|   Carolyn|  5.98|2007-05-14 13:44:...|
|         14|     Betty|  4.99|2007-05-14 13:44:...|
|         43| Christine|  0.00|2007-05-14 13:44:...|
|         15|     Helen|  3.98|2007-05-14 13:44:...|
|         43| Christine|  3.98|2007-05-14 13:44:...|
|         22|     Laura|  4.99|2007-05-14 13:44:...|
|         44|     Marie|  4.99|2007-05-14 13:44:...|
|         28|   Cynthia|  2.99|2007-05-14 13:44:...|
|         29|    Angela|  0.99|2007-05-14 13:44:...|
|         33|      Anna|  0.99|2007-05-14 13:44:...|
|         60|   Mildred|  9.98|2007-05-14 13:4

# INNER JOIN with an Alias

In [9]:
def spark_join(df1,df2,on: list = [],how: str = 'inner'):
    if not on:
        on = list(set(df1.columns).intersection(set(df2.columns)))
    return df1.join(df2,on = on, how = how)

In [10]:
query1 = """SELECT
	customer_id,
	first_name,
    last_name
FROM
	customer
"""
query2 = """SELECT customer_id,staff_id,amount,payment_date FROM payment"""
query3 = """SELECT staff_id,first_name AS staff_first_name,last_name AS staff_last_name
FROM staff"""
df1 = spark.sql(query1)
df2 = spark.sql(query2)
df3 = spark.sql(query3)

In [11]:
df_joined = reduce(spark_join,[df1,df2,df3]).orderBy('payment_date')

In [12]:
df_joined.show()

+--------+-----------+----------+---------+------+--------------------+----------------+---------------+
|staff_id|customer_id|first_name|last_name|amount|        payment_date|staff_first_name|staff_last_name|
+--------+-----------+----------+---------+------+--------------------+----------------+---------------+
|       2|        416|   Jeffery|   Pinson|  2.99|2007-02-14 21:21:...|             Jon|       Stephens|
|       2|        516|     Elmer|      Noe|  4.99|2007-02-14 21:23:...|             Jon|       Stephens|
|       1|        239|    Minnie|   Romero|  4.99|2007-02-14 21:29:...|            Mike|        Hillyer|
|       2|        592|  Terrance|    Roush|  6.99|2007-02-14 21:41:...|             Jon|       Stephens|
|       1|         49|     Joyce|  Edwards|  0.99|2007-02-14 21:44:...|            Mike|        Hillyer|
|       2|        264| Gwendolyn|      May|  3.99|2007-02-14 21:44:...|             Jon|       Stephens|
|       1|         46| Catherine| Campbell|  4.99|2007-

# LEFT JOIN

In [13]:
query1 = """SELECT film_id,
                   title
            FROM film
"""
query2 = """SELECT film_id,inventory_id FROM inventory"""
df1 = spark.sql(query1)
df2 = spark.sql(query2)
df_join = spark_join(df1,df2,how = 'left',on = ['film_id']).orderBy('title')

In [14]:
df_join.show()

+-------+----------------+------------+
|film_id|           title|inventory_id|
+-------+----------------+------------+
|      1|Academy Dinosaur|           1|
|      1|Academy Dinosaur|           8|
|      1|Academy Dinosaur|           7|
|      1|Academy Dinosaur|           6|
|      1|Academy Dinosaur|           5|
|      1|Academy Dinosaur|           4|
|      1|Academy Dinosaur|           3|
|      1|Academy Dinosaur|           2|
|      2|  Ace Goldfinger|          11|
|      2|  Ace Goldfinger|          10|
|      2|  Ace Goldfinger|           9|
|      3|Adaptation Holes|          13|
|      3|Adaptation Holes|          12|
|      3|Adaptation Holes|          15|
|      3|Adaptation Holes|          14|
|      4|Affair Prejudice|          22|
|      4|Affair Prejudice|          21|
|      4|Affair Prejudice|          20|
|      4|Affair Prejudice|          19|
|      4|Affair Prejudice|          18|
+-------+----------------+------------+
only showing top 20 rows



# RIGHT JOIN

In [15]:
film_schema = StructType([ \
    StructField("film_id",IntegerType(),True), \
    StructField("title",StringType(),True)
  ])
films = [(1,'Joker'),
            (2,'Avengers'),
            (3,'Parasite')]
columns = ['film_id','title']
rdd = spark.sparkContext.parallelize(films)
df1 = spark.createDataFrame(rdd, schema = film_schema)
ratings_schema = StructType([ \
    StructField("review_id",IntegerType(),True),
    StructField("film_id",IntegerType(),True),
    StructField("review",StringType(),True),
  ])
ratings = [(1,1,'Excellent'),
         (2,1,'Awesome'),
         (3,2,'Cool'),
         (4,None,'Beautiful')]
columns = ['review_id','film_id','review']
rdd = spark.sparkContext.parallelize(ratings)
df2 = spark.createDataFrame(rdd, schema = ratings_schema)

In [16]:
df_join = spark_join(df1,df2,how = 'right',on = ['film_id']).select(['review','title'])

In [17]:
df_join.show()

+---------+--------+
|   review|   title|
+---------+--------+
|Excellent|   Joker|
|  Awesome|   Joker|
|     Cool|Avengers|
|Beautiful|    null|
+---------+--------+



## Filtering example

In [18]:
df_join.filter(df_join['title'].isNull()).show()

+---------+-----+
|   review|title|
+---------+-----+
|Beautiful| null|
+---------+-----+



# Turning a RIGHT JOIN into a LEFT JOIN

In [19]:
df_join = spark_join(df2,df1,how = 'left',on = ['film_id']).select(['review','title'])

In [20]:
df_join.show()

+---------+--------+
|   review|   title|
+---------+--------+
|Excellent|   Joker|
|  Awesome|   Joker|
|     Cool|Avengers|
|Beautiful|    null|
+---------+--------+



# SELF JOIN

In [21]:
company_schema = StructType([ \
    StructField("employee_id",IntegerType(),True),
    StructField("first_name",StringType(),True),
    StructField("last_name",StringType(),True),
    StructField("manager_id",IntegerType(),True),
  ])
employees = [(1, 'Windy', 'Hays', None),
             (2, 'Ava', 'Christensen', 1),
             (3, 'Hassan', 'Conner', 1),
             (4, 'Anna', 'Reeves', 2),
             (5, 'Sau', 'Norman', 2),
             (6, 'Kelsie', 'Hays', 3),
             (7, 'Tory', 'Goff', 3),
             (8, 'Salley', 'Lester', 3)]

rdd = spark.sparkContext.parallelize(employees)
df1 = spark.createDataFrame(rdd, schema = company_schema)
df1 = df1.select('employee_id','manager_id',F.concat_ws(' ', df1['first_name'], df1['last_name']).alias('employee_full_name'))

## query example

In [22]:
df_join = df1.alias('t1')\
             .join(df1.alias('t2'),
                   F.col('t1.employee_id') == F.col('t2.manager_id'),
                   'inner')\
            .select(*[
                F.col('t1.employee_full_name').alias('manager'),
                F.col('t2.employee_full_name').alias('employee')
            ]).orderBy('manager')

In [23]:
df_join.show()

+---------------+---------------+
|        manager|       employee|
+---------------+---------------+
|Ava Christensen|    Anna Reeves|
|Ava Christensen|     Sau Norman|
|  Hassan Conner|    Kelsie Hays|
|  Hassan Conner|      Tory Goff|
|  Hassan Conner|  Salley Lester|
|     Windy Hays|Ava Christensen|
|     Windy Hays|  Hassan Conner|
+---------------+---------------+



## Showing who's the boss

In [24]:
df_join = df1.alias('e')\
             .join(df1.alias('m'),
                   F.col('m.employee_id') == F.col('e.manager_id'),
                   'left')\
            .select(*[
                F.col('e.employee_full_name').alias('employee'),
                F.col('m.employee_full_name').alias('manager')
                
            ]).orderBy('manager')

In [25]:
df_join.show()

+---------------+---------------+
|       employee|        manager|
+---------------+---------------+
|     Windy Hays|           null|
|    Anna Reeves|Ava Christensen|
|     Sau Norman|Ava Christensen|
|    Kelsie Hays|  Hassan Conner|
|      Tory Goff|  Hassan Conner|
|  Salley Lester|  Hassan Conner|
|Ava Christensen|     Windy Hays|
|  Hassan Conner|     Windy Hays|
+---------------+---------------+



# FULL OUTER JOIN

In [26]:
departments_schema = StructType([ \
    StructField("department_id",IntegerType(),True),
    StructField("department_name",StringType(),True)])
employee_schema = StructType([ \
    StructField("employee_id",IntegerType(),True),
    StructField("employee_name",StringType(),True),
    StructField("department_id",IntegerType(),True)])
departments = [(1, 'Sales'),
               (2, 'Marketing'),
               (3, 'HR'),
               (4, 'IT'),
               (5,'Production')]

rdd = spark.sparkContext.parallelize(departments)
df1 = spark.createDataFrame(rdd, schema = departments_schema)
employees = [(1,'Bette Nicholson', 1),
        	 (2,'Christian Gable', 1),
        	 (3,'Joe Swank', 2),
        	 (4,'Fred Costner', 3),
        	 (5,'Sandra Kilmer', 4),
        	 (6,'Julia Mcqueen', None)]
rdd = spark.sparkContext.parallelize(employees)
df2 = spark.createDataFrame(rdd, schema = employee_schema)

In [27]:
spark_join(df1,df2,how = 'full_outer').select('employee_name','department_name').show()

+---------------+---------------+
|  employee_name|department_name|
+---------------+---------------+
|  Julia Mcqueen|           null|
|Christian Gable|          Sales|
|Bette Nicholson|          Sales|
|      Joe Swank|      Marketing|
|   Fred Costner|             HR|
|  Sandra Kilmer|             IT|
|           null|     Production|
+---------------+---------------+



In [28]:
df_join = df2.select(F.col('employee_name').alias('employee_A')).crossJoin(df2.select(F.col('employee_name').alias('employee_B')))
df_join = df_join.filter(F.col('employee_A') != F.col('employee_B'))

In [29]:
df_join.show()

+---------------+---------------+
|     employee_A|     employee_B|
+---------------+---------------+
|Bette Nicholson|Christian Gable|
|Bette Nicholson|      Joe Swank|
|Bette Nicholson|   Fred Costner|
|Bette Nicholson|  Sandra Kilmer|
|Bette Nicholson|  Julia Mcqueen|
|Christian Gable|Bette Nicholson|
|Christian Gable|      Joe Swank|
|Christian Gable|   Fred Costner|
|Christian Gable|  Sandra Kilmer|
|Christian Gable|  Julia Mcqueen|
|      Joe Swank|Bette Nicholson|
|      Joe Swank|Christian Gable|
|      Joe Swank|   Fred Costner|
|      Joe Swank|  Sandra Kilmer|
|      Joe Swank|  Julia Mcqueen|
|   Fred Costner|Bette Nicholson|
|   Fred Costner|Christian Gable|
|   Fred Costner|      Joe Swank|
|   Fred Costner|  Sandra Kilmer|
|   Fred Costner|  Julia Mcqueen|
+---------------+---------------+
only showing top 20 rows



In [30]:
df_join = df1.select(F.col('department_name').alias('department_A')).crossJoin(df1.select(F.col('department_name').alias('department_B')))
df_join = df_join.filter(F.col('department_A') != F.col('department_B'))

In [31]:
df_join.show()

+------------+------------+
|department_A|department_B|
+------------+------------+
|       Sales|   Marketing|
|       Sales|          HR|
|       Sales|          IT|
|       Sales|  Production|
|   Marketing|       Sales|
|   Marketing|          HR|
|   Marketing|          IT|
|   Marketing|  Production|
|          HR|       Sales|
|          HR|   Marketing|
|          HR|          IT|
|          HR|  Production|
|          IT|       Sales|
|          IT|   Marketing|
|          IT|          HR|
|          IT|  Production|
|  Production|       Sales|
|  Production|   Marketing|
|  Production|          HR|
|  Production|          IT|
+------------+------------+

